In [ ]:
# pip install pandas tqdm jaydebeapi jpype openpyxl
import pandas as pd
import geopandas as gpd
import shapely
from shapely import wkt
from tqdm import tqdm
import jaydebeapi as jdbc
import jpype
import os



In [ ]:
#bbox to obtain buildings
min_longitude = 39.57001575674445
max_longitude = 39.69913024255342
min_latitude = -4.090533801671711
max_latitude = -4.000157637723547

In [ ]:
#!!!!!!change name of table accordingly
sql_tablename = "FEATURES_DB_VIDA_EXTENDED"

# connect to the IBM DB2 function
def connect_to_db():

    jar = "db2jcc4.jar"
    os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-24\\bin\server"
    os.environ["CLASSPATH"] = jar

    args = "-Djava.class.path=%s" % jar
    jvm_path = jpype.getDefaultJVMPath()
    try:
        jpype.startJVM(jvm_path, args, convertStrings=True)
    except Exception as e:
        # print('startJVM exception: ', e)
        pass

    if jpype.isJVMStarted() and not jpype.isThreadAttachedToJVM():
        jpype.attachThreadToJVM()
        jpype.java.lang.Thread.currentThread().setContextClassLoader(
            jpype.java.lang.ClassLoader.getSystemClassLoader()
        )

    username = "xxxxxx"
    password = "xxxxxxxx"
    # connstr = 'jdbc:db2://<host><port>/BLUDB:sslConnection=true;sslTrustStorePassword=trustMe;sslTrustStoreLocation=<full path to jks file>;useJDBC4ColumnNameAndLabelSemantics=false;db2.jcc.charsetDecoderEncoder=3;'
    drvrpth = "db2jcc4.jar"
    drvrnm = "com.ibm.db2.jcc.DB2Driver"

    connstr = "jdbc:db2://65beb513-5d3d-4101-9001-f42e9dc954b3.brt9d04f0cmqeb8u7740.databases.appdomain.cloud:30371/BLUDB:sslConnection=true;useJDBC4ColumnNameAndLabelSemantics=false;db2.jcc.charsetDecoderEncoder=3;"

    dw_active = "yes"
    dm_active = "yes"

    host = "65beb513-5d3d-4101-9001-f42e9dc954b3.brt9d04f0cmqeb8u7740.databases.appdomain.cloud:30371"
    jdbc_driver_name = "com.ibm.db2.jcc.DB2Driver"
    jdbc_driver_loc = "db2jcc4.jar"

    dw_active = "yes"
    dm_active = "yes"

    conn = jdbc.connect(drvrnm, connstr, [username, password], drvrpth)

    return conn


def fetch_builings(cursor, max_lon, min_lon, min_lat, max_lat):
    """
    This particular function is aimed for obtating all entries from defined rectangle for selected SQL table
    """
    # select of parameters from DB
    columns = ["ID", "LATITUDE", "LONGITUDE", "VIDA_CONFIDENCE", "AREA_IN_METERS", "PERIMETER_IN_METERS", "OSM_TYPE", "POLYGON_COORDINATES", "GHSL_SMOD"]

    sql = f"""
        SELECT {', '.join(columns)}
        FROM USER1.{sql_tablename}
        WHERE
            LONGITUDE <= {max_lon} AND
            LONGITUDE >= {min_lon} AND
            LATITUDE >= {min_lat} AND
            LATITUDE <= {max_lat}
    """

    try:
        cursor.execute(sql)
        data = cursor.fetchall()
    except Exception as e:
        print(f"Fetch items error occurred: {e}")
        print("Reconnecting to the database try again...")

        conn = connect_to_db()
        cursor = conn.cursor()
        cursor.execute(sql)
        data = cursor.fetchall()
    finally:
        # # reshape obtained data to the GeoDataFrame
        # df = pd.DataFrame(data=data, columns=columns)
        # df = gpd.GeoDataFrame(
        #     df, geometry=shapely.from_wkt(df.polygon_coordinates.astype(str))
        # )
        # df = df.drop(["polygon_coordinates"], axis=1)
        return data

In [ ]:
conn = connect_to_db()
cursor = conn.cursor()

rows = []

# Get data from database using the longitude and latitude
data_from_db = fetch_builings(
    cursor,
    max_lon=max_longitude,
    min_lon=min_longitude,
    min_lat=min_latitude,
    max_lat=max_latitude,
)

data = data_from_db

for item in data:
    id, latitude, longitude, vida_confidence, area_in_meters, perimeter_in_meters, osm_type, polygon_coordinates, ghsl_smod = item
    # Create a dictionary for the row with parameters from DB

    db_row = {
        "id": id,
        "latitude": f"{latitude}",
        "longitude": f"{longitude}",
        "vida_confidence": vida_confidence,
        "area_in_meters": area_in_meters,
        "perimeter_in_meters": perimeter_in_meters,
        "osm_type": osm_type,
        "polygon_coordinates": polygon_coordinates,
        "ghsl_smod": ghsl_smod,
    }

    # Append the row to the list
    rows.append(db_row)

# Close the cursor and connection when done
cursor.close()
conn.close()

df = pd.DataFrame(data=rows)

df = gpd.GeoDataFrame(
            df, geometry=shapely.from_wkt(df.polygon_coordinates.astype(str))
        )

df.to_parquet("all_for_training_db.parquet")
print(df.head(3))

In [ ]:
df["id"].count()

In [ ]:
df.columns